In [1]:
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk
import pandas
from tqdm import tqdm

In [2]:
dataset_name = "presencesw/vinli_3_label"
label = 'neutral'
dataset = load_dataset(dataset_name)

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 18282
    })
    test: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2264
    })
    dev: Dataset({
        features: ['gold_label', 'sentence1', 'sentence2'],
        num_rows: 2255
    })
})


In [4]:
def convert_entailment(exmaples):
    labels = exmaples['gold_label']
    labels = [f"not_{label}" if i != label else label for i in labels]
    exmaples['gold_label'] = labels
    return exmaples

In [5]:
# this step convert dataset to 2 labels
converted_dataset = dataset.map(convert_entailment, batched=True)

# check number samples in dataset train
converted_dataset['train']

Dataset({
    features: ['gold_label', 'sentence1', 'sentence2'],
    num_rows: 18282
})

In [6]:
# Divine the dataset to 2 sets `entailment` and `not entailment`
new_dataset_entail = converted_dataset.filter(lambda example: example['gold_label'] == label)['train']
new_dataset_not_entail = converted_dataset.filter(lambda example: example['gold_label'] == f'not_{label}')['train']

In [7]:
# convert dataset to pandas for next step
new_dataset_entail = Dataset.to_pandas(new_dataset_entail)
# duplicate dataset not entailment
new_dataset_not_entail1 = Dataset.to_pandas(new_dataset_not_entail)
new_dataset_not_entail2 = Dataset.to_pandas(new_dataset_not_entail)

In [8]:
# merge 2 dataset not entailment for the target
j = 0
for i in tqdm(range(len(new_dataset_not_entail1))):
    temp = set()
    # check = False
    # for j in range(len(new_dataset_not_entail2)):
    for index in range(100):
        new_j = j + index
        if new_j == len(new_dataset_not_entail1):
            j = new_j
            break
        if new_dataset_not_entail1.iloc[i]['sentence1'] != new_dataset_not_entail2.iloc[new_j]['sentence1']:
            j = new_j
            break
        temp.add(new_dataset_not_entail2.iloc[new_j]['sentence2'])
    # new_dataset_not_entail1.iloc[i]['sentence2'] = temp
    # temp = [val for val in temp]
    new_dataset_not_entail1.at[i, 'sentence2'] = temp

100%|██████████| 12188/12188 [00:01<00:00, 6416.55it/s]


In [9]:
# dataset_not_entail = new_dataset_not_entail1.drop_duplicates(subset=['sentence2'])
# print(len(dataset_not_entail))
dataset_not_entail = new_dataset_not_entail1[new_dataset_not_entail1['sentence2'] != set()]
print(len(dataset_not_entail))

3044


In [10]:
dataset_not_entail = dataset_not_entail.reset_index(drop=True)
dataset_not_entail

,gold_label,sentence1,sentence2
0,not_neutral,"Sau dự án điện mặt trời, Tập đoàn Trung Nam vừ...",{Tập đoàn Trung Nguyên vừa bán hơn hơn 30% cổ ...
1,not_neutral,"Phía Tập đoàn Trung Nam cho biết, với 64,9% cổ...",{Tập đoàn Trung Nam nắm giữ phần lớn lượng cổ ...
2,not_neutral,"Cách đây một tháng, tập đoàn này cũng bán 49% ...",{Vốn của một công ty nước ngoài đang chiếm 49%...
3,not_neutral,"Dự án Nhà máy Điện gió Trung Nam, tổng số vốn ...",{Sản lượng điện dự kiến tạo ra trong một năm t...
4,not_neutral,"Từ nay tới năm 2028, dòng vốn đầu tư được dự b...",{Nguyên nhân cho việc dòng vốn đầu tư từ nay c...
...,...,...,...
3039,not_neutral,Ổ SSD dùng để khai thác tiền số Chia chỉ còn t...,{Dùng ổ cứng SSD để khai thác tiền số Chia làm...
3040,not_neutral,"Khi khai thác Chia, những ổ cứng dung lượng lớ...",{Các ổ cứng 512 GB sẽ có tuổi thọ cao hơn so v...
3041,not_neutral,Ổ cứng thường được hãng bảo hành từ ba đến năm...,{Người dùng chỉ được bảo hành trong 1 năm sau ...
3042,not_neutral,"Ổ SSD được ""thợ đào"" Chia yêu thích vì tốc độ ...",{Nhiều người dùng ổ cứng SSD để khai thác tiền...


In [11]:
temp = []
for index1 in tqdm(range(len(new_dataset_entail))):
    # print(new_dataset_entail.iloc[index1]['sentence1'])
    for index2 in range(len(dataset_not_entail)):
        if new_dataset_entail.iloc[index1]['sentence1'] == dataset_not_entail.iloc[index2]['sentence1']:
            temp.append(dataset_not_entail.iloc[index2]['sentence2'])
            break

  0%|          | 0/6094 [00:00<?, ?it/s]

100%|██████████| 6094/6094 [04:28<00:00, 22.73it/s]


In [12]:
print(len(temp))

6094


In [13]:
new_dataset_entail['negative'] = temp
# new_dataset_entail

In [14]:
new_dataset = Dataset.from_pandas(new_dataset_entail)
new_dataset = new_dataset.rename_column("sentence1", "anchor")
new_dataset = new_dataset.rename_column("sentence2", "positive")
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 6094
})

In [15]:
def convert_dataset(example):
    # print(example['negative'])
    number = len(example['negative'][0])
    # print(gold_label)
    gold_label = example['gold_label'] * number
    anchor = example['anchor'] * number
    positive = example['positive'] * number
    return {'gold_label': gold_label, 'anchor': anchor, 
            'positive': positive, 'negative': example['negative'][0]}

        
new_dataset = new_dataset.map(
    lambda example: convert_dataset(example),                   
    remove_columns=['gold_label', 'anchor', 'positive', 'negative'],
    batched=True,
    batch_size=1
)

Map:   0%|          | 0/6094 [00:00<?, ? examples/s]

In [16]:
new_dataset

Dataset({
    features: ['gold_label', 'anchor', 'positive', 'negative'],
    num_rows: 24406
})

In [30]:
print(['']*5)

['', '', '', '', '']


In [31]:
def add_columns(examples):
    # print(len(examples))
    length = len(examples['gold_label'])
    examples['negative'] = [''] * length
    return examples

In [17]:
converted_dataset = converted_dataset.rename_column("sentence1", "anchor")
converted_dataset = converted_dataset.rename_column("sentence2", "positive")

In [35]:
converted_dataset = converted_dataset.map(add_columns, batched=True)

In [36]:
all_data = DatasetDict()
all_data['train'] = new_dataset
all_data['validation'] = converted_dataset['dev']
all_data['test'] = converted_dataset['test']

In [37]:
all_data.push_to_hub(f"{dataset_name}_{label}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/presencesw/vinli_3_label_neutral/commit/0b546ea05a84bfa5ee851a026be5daeac70b9468', commit_message='Upload dataset', commit_description='', oid='0b546ea05a84bfa5ee851a026be5daeac70b9468', pr_url=None, pr_revision=None, pr_num=None)